# Project

## Dépendance

In [1]:
! pip install SPARQLWrapper
! pip install exifread
! pip install pandas
! pip install PIL

DEPRECATION: Loading egg at c:\users\guigu\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\users\guigu\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\users\guigu\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\guigu\appdata\local\programs\python\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


## Collecte de données

In [2]:
import pandas as pd
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# Get cities
query = """SELECT DISTINCT ?grandeville ?grandevilleLabel ?pays ?paysLabel ?image {
  ?grandeville wdt:P31 wd:Q1549591;
               wdt:P17 ?pays;
               wdt:P18 ?image.
 SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }
}
LIMIT 10"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (
        sys.version_info[0],
        sys.version_info[1],
    )
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


array = []
results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    array.append(
        (
            result["grandevilleLabel"]["value"],
            result["paysLabel"]["value"],
            result["image"]["value"],
        )
    )

dataframe = pd.DataFrame(array, columns=["ville", "pays", "image"])
dataframe = dataframe.astype(
    dtype={"ville": "<U200", "pays": "<U200", "image": "<U200"}
)
dataframe

,ville,pays,image
0,Saratov,Russie,http://commons.wikimedia.org/wiki/Special:File...
1,Thimphou,Bhoutan,http://commons.wikimedia.org/wiki/Special:File...
2,Bichkek,Kirghizistan,http://commons.wikimedia.org/wiki/Special:File...
3,Glasgow,Royaume-Uni,http://commons.wikimedia.org/wiki/Special:File...
4,Shanghai,Chine,http://commons.wikimedia.org/wiki/Special:File...
5,Las Piñas,Philippines,http://commons.wikimedia.org/wiki/Special:File...
6,Samarcande,Ouzbékistan,http://commons.wikimedia.org/wiki/Special:File...
7,Sébastopol,Russie,http://commons.wikimedia.org/wiki/Special:File...
8,Orléans,France,http://commons.wikimedia.org/wiki/Special:File...
9,Oulan-Oudé,Russie,http://commons.wikimedia.org/wiki/Special:File...


### Téléchargement des images

In [3]:
import requests
import shutil
import os

def download_image(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    request = requests.get(url, allow_redirects=True, headers=headers, stream=True)
    if request.status_code == 200:
        # Create 'images' directory if it doesn't exist
        if not os.path.exists("images"):
            os.makedirs("images")

        # Extract the filename from the URL and save the image in 'images' directory
        filename = os.path.join("images", os.path.basename(url))
        with open(filename, "wb") as image:
            request.raw.decode_content = True
            shutil.copyfileobj(request.raw, image)
    return request.status_code

def clear_images_directory():
    # Clear 'images' directory if it exists
    if os.path.exists("images"):
        shutil.rmtree("images")

# Clear 'images' directory before every execution
clear_images_directory()

# Assuming 'dataframe' is a DataFrame containing URLs under column 'image'
# Apply the download_image function to each URL in the 'image' column
dataframe.image.apply(download_image)

0    200
1    200
2    200
3    200
4    200
5    200
6    200
7    200
8    200
9    200
Name: image, dtype: int64

### Enregistrement des métadonnées
#### Import

In [4]:
import os
import json
from PIL import Image, TiffImagePlugin
import PIL.ExifTags

#### Define the directory path and initialize the list to store metadata:

In [5]:
directory_path = "images"
all_metadata = []

if not os.path.isdir(directory_path):
    print(f"Error: Directory '{directory_path}' does not exist!")
    sys.exit()

#### Define the function to cast data types

In [6]:
def cast(v):
    if isinstance(v, TiffImagePlugin.IFDRational):
        if v.denominator == 0:
            return None  # Handle division by zero gracefully
        return float(v.numerator) / float(v.denominator)
    elif isinstance(v, tuple):
        return tuple(cast(t) for t in v)
    elif isinstance(v, bytes):
        return v.decode(errors="replace")
    elif isinstance(v, dict):
        for kk, vv in v.items():
            v[kk] = cast(vv)
        return v
    else:
        return v

#### Define the function to get the images properties

In [7]:
def get_image_metadata(imgfile, hasExif):
    metadata = {}

    # Get image format
    metadata['format'] = imgfile.format

    # Get image size
    metadata['size'] = imgfile.size
    
    if hasExif:
        exif = dict(imgfile._getexif().items())
        if exif:
            # Get image orientation (landscape, portrait, square, etc.)
            if 274 in exif:
                orientation = exif[274]
                if orientation == 1:
                    metadata['orientation'] = 'Landscape'
                elif orientation == 3:
                    metadata['orientation'] = 'Portrait'
                else:
                    metadata['orientation'] = 'Unknown'
            else:
                metadata['orientation'] = 'Unknown'
    else:
        metadata['orientation'] = 'Unknown'
    return metadata

#### Loop through files in the directory and extract metadata

In [8]:
import os
import json
import sys
from PIL import Image, TiffImagePlugin
import PIL.ExifTags

directory_path = "images"
all_metadata = []

if not os.path.isdir(directory_path):
    print(f"Error: Directory '{directory_path}' does not exist!")
    sys.exit()

for filename in os.listdir(directory_path):
    if os.path.isfile(os.path.join(directory_path, filename)) and filename.lower().endswith((".jpg", ".png")):
        print(f"Found image: {filename}")

        try:
            imgfile = Image.open(os.path.join(directory_path, filename))
            exif_data = imgfile._getexif()
            
            metadata_dict = {}
            
            hasExif = False

            if exif_data:
                hasExif = True
                for k, v in imgfile._getexif().items():
                    if k in PIL.ExifTags.TAGS:
                        v = cast(v)
                        metadata_dict[PIL.ExifTags.TAGS[k]] = v
            else:
                print(f"  - No EXIF data found for {filename}")

            # Get additional image metadata (format, size, orientation)
            image_metadata = get_image_metadata(imgfile, hasExif)
            metadata_dict.update(image_metadata)
            
            all_metadata.append({filename: metadata_dict})
            if not metadata_dict:
                print(f"  - No metadata found for {filename}")

        except Exception as e:
            print(f"  - Error processing {filename}: {e}")

directory_path = "output"
output_file = "directory_metadata.json"

if not os.path.exists(directory_path):
            os.makedirs(directory_path)

output_location = directory_path + "/" + output_file
with open(output_location, "w") as json_file:
    json.dump(all_metadata, json_file, indent=4)

print(f"Directory metadata saved to {output_location} (if no errors occurred)")


Found image: %D0%9A%D1%80%D1%8B%D1%82%D1%8B%D0%B9%20%D1%80%D1%8B%D0%BD%D0%BE%D0%BA.%20%D0%A1%D0%B0%D1%80%D0%B0%D1%82%D0%BE%D0%B2.jpg
Found image: Barangay%20Talon%20Las%20Pinas%20City%20Aerial%20Photo.jpg
Found image: Bishkek.jpg
Found image: Orl%C3%A9ans%2C%20l%27automne%20est%20l%C3%A0%20%21.jpg
Found image: Samarqand.jpg
  - No EXIF data found for Samarqand.jpg
Found image: Sevastopol%20Collage%202015.png
  - No EXIF data found for Sevastopol%20Collage%202015.png
Found image: Shanghai%20121.45796E%2031.22234N.jpg
  - No EXIF data found for Shanghai%20121.45796E%2031.22234N.jpg
Found image: Thimphu-12-vom%20Buddha%20Dordenma-2015-gje.jpg
Found image: Ulan-Ude%20Lenina%20st-2.JPG
Found image: Wfm%20foster%20armadillo.jpg
Directory metadata saved to output/directory_metadata.json (if no errors occurred)


In [ ]:
! pip install ipywidgets

In [14]:


import ipywidgets as widgets

from os import listdir

images = []

for file in listdir("./images"):
    if (".png" in file) or (".jpg" in file):
        images.append("./images/" + file)
        
from ipywidgets import GridspecLayout, Image, interact
paths = []

checkboxes = [widgets.Checkbox(value=False, description='Favorite') for _ in range(len(images))]
tags = [widgets.Text(value='', placeholder='Type something', description='String:', disabled=False) for _ in range (len(images))]

# Create the GridspecLayout widget
layout = GridspecLayout(n_columns=2, n_rows=len(images), width='400px')
for i, (img, tag) in enumerate(zip(images, tags)):
  file = open(img, "rb")
  image = file.read()
  image_widget = widgets.Image(
    value=image,
    format=('png' or 'jpg'),
    width=100,
    height=100,
  )
  layout[i,0] = image_widget
  layout[i, 1] = tag

# Button to get selected images
button = widgets.Button(description="Select")

# Output widget to display selected images
output = widgets.Output()

# Function to get selected images
def get_selected_images(btn):
    global paths
    paths = []
    selected_paths = [images[i] for i, tag in enumerate(tags) if tag.value]
    with output:
        output.clear_output()
        print("Selected Images:")
        for path in selected_paths:
            print(path)
            paths.append(path)
    print(paths)
        

# Link button click event to function
button.on_click(get_selected_images)

# Display the layout and button
display(layout, button, output)


GridspecLayout(children=(Image(value=b'\xff\xd8\xff\xe1\x00\xa1Exif\x00\x00II*\x00\x08\x00\x00\x00\x05\x00\x12…

Button(description='Select', style=ButtonStyle())

Output()

['./images/%D0%9A%D1%80%D1%8B%D1%82%D1%8B%D0%B9%20%D1%80%D1%8B%D0%BD%D0%BE%D0%BA.%20%D0%A1%D0%B0%D1%80%D0%B0%D1%82%D0%BE%D0%B2.jpg', './images/Barangay%20Talon%20Las%20Pinas%20City%20Aerial%20Photo.jpg', './images/Bishkek.jpg', './images/Orl%C3%A9ans%2C%20l%27automne%20est%20l%C3%A0%20%21.jpg', './images/Samarqand.jpg', './images/Sevastopol%20Collage%202015.png', './images/Shanghai%20121.45796E%2031.22234N.jpg', './images/Thimphu-12-vom%20Buddha%20Dordenma-2015-gje.jpg', './images/Wfm%20foster%20armadillo.jpg']


In [12]:
images

['./images/Sevastopol%20Collage%202015.png']